In [40]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta
from SML.training_pipeline import model_dir
import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [41]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [42]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-05-12


In [43]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-05-13


In [44]:
feature_view = fs.get_feature_view('tesla_stocks_fv', 5)
feature_view.init_batch_scoring(training_dataset_version=1)

In [45]:
print(feature_view.get_batch_query())

WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`ticker` `ticker`, `fg1`.`ticker` `join_pk_ticker`, `fg1`.`date` `join_evt_date`, `fg0`.`sentiment` `sentiment`, RANK() OVER (PARTITION BY `fg1`.`ticker`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `klittefr_featurestore`.`tesla_stock_5` `fg1`
INNER JOIN `klittefr_featurestore`.`news_sentiment_updated_5` `fg0` ON `fg1`.`ticker` = `fg0`.`ticker` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`ticker` `ticker`, `right_fg0`.`sentiment` `sentiment`
FROM right_fg0)


In [46]:
# we had problems fetching the data from fv with get_batch_data function, tried everything and it just doesnt work 
tesla_df_b = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.20s) 


In [47]:
tesla_df_b.head()

,date,ticker,sentiment
0,2024-05-13 00:00:00+00:00,TSLA,0.115443


In [48]:
from sklearn.preprocessing import OneHotEncoder

In [49]:
#OneHotEncoding the tesla_df_b column 'ticker'

tickers = tesla_df_b[['ticker']]

# Initializing OneHotEncoder
encoder = OneHotEncoder()

# Fitting and transforming the 'ticker' column
ticker_encoded_test = encoder.fit_transform(tickers)

# Converting the encoded column into a DataFrame
ticker_encoded_df_test = pd.DataFrame(ticker_encoded_test.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenating the encoded DataFrame with the original DataFrame
tesla_df_b = pd.concat([tesla_df_b, ticker_encoded_df_test], axis=1)

# Dropping the original 'ticker' column
tesla_df_b.drop('ticker', axis=1, inplace=True)

In [50]:
# As X_train['date'] column exists and is in datetime format, we're converting it
tesla_df_b['year'] = tesla_df_b['date'].dt.year
tesla_df_b['month'] = tesla_df_b['date'].dt.month
tesla_df_b['day'] = tesla_df_b['date'].dt.day

# Dropping the original date column
tesla_df_b.drop(columns=['date'], inplace=True)

# Converting dataframe to numpy array
tesla_df_b_array = tesla_df_b.to_numpy()

# Reshaping the array to have a shape suitable for LSTM
tesla_df_b_array = np.expand_dims(tesla_df_b_array, axis=1)

2024-05-14 12:30:49,197 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [51]:
import joblib

the_model = mr.get_model("stock_pred_model", version=28)
model_dir = the_model.download()

model = joblib.load(model_dir + "/stock_prediction_model.pkl")

In [52]:
predictions = model.predict(tesla_df_b_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [53]:
predictions 

array([[0.8625609]], dtype=float32)

In [54]:
import numpy as np

# Our predictions array
predictions = np.array(predictions, dtype=np.float32)

# Changing the format of the predicted value to correspond with format of "open"
predictions = predictions[0][0]*100
print(predictions)


86.25609278678894


In [55]:
tesla_df_b['predictions'] = predictions.tolist()

In [56]:
# Assuming you have 'year', 'month', and 'day' columns in your DataFrame
tesla_df_b['date'] = pd.to_datetime(tesla_df_b[['year', 'month', 'day']])

# Now you can drop the 'year', 'month', and 'day' columns if you want
tesla_df_b.drop(columns=['year', 'month', 'day'], inplace=True)

In [57]:
tesla_df_b['date'] = pd.to_datetime(tesla_df_b['date'])

In [58]:
tesla_df_b

,sentiment,ticker_TSLA,predictions,date
0,0.115443,1.0,86.256093,2024-05-13


In [59]:
# Convert the encoded DataFrame back to numpy array
ticker_encoded_array = ticker_encoded_df_test.to_numpy()

# Inverse transform the encoded array to retrieve the original values
original_tickers = encoder.inverse_transform(ticker_encoded_array)

# Convert the original_tickers array to a DataFrame
original_tickers_df = pd.DataFrame(original_tickers, columns=['ticker'])

# Concatenate the original ticker column with the remaining columns from tesla_df_b
tesla_df_b = pd.concat([tesla_df_b.drop(columns=['ticker_TSLA']), original_tickers_df], axis=1)


In [60]:
tesla_df_b.head()

,sentiment,predictions,date,ticker
0,0.115443,86.256093,2024-05-13,TSLA


In [61]:
#from sklearn.preprocessing import MinMaxScaler

# Flatten the list of lists into a single list
#flat_predictions_scaled = [item for sublist in predictions_scaled for item in sublist]

# Initialize the MinMaxScaler
#scaler = MinMaxScaler()

# Fit the scaler to the scaled predictions
#scaler.fit(flat_predictions_scaled)

# Inverse transform the scaled predictions to get the original values
#predictions_unscaled = scaler.inverse_transform(flat_predictions_scaled)

# Update the 'predictions' column with the unscaled values
#tesla_df_b['predictions'] = predictions_unscaled

In [62]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [65]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 4,
    description = 'Predction of TSLA open stock price',
    primary_key = ['ticker'],
    event_time = ['date'],
    online_enabled = False,
)

2024-05-14 12:39:44,585 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [66]:
#Inserting the stock data into the stocks feature group
results_fg.insert(tesla_df_b, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/814414


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/stock_prediction_results_4_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x23f71193dd0>, None)